In [19]:
# 注意：尽量展示所有环节和过程中出现过的所有问题

In [4]:
import numpy as np
numbers=[73,60,62.1,37.8]
print(np.mean(numbers))

58.224999999999994


In [6]:
# 调用bart-base-chinese预训练模型，LCSTS数据集，加以微调
# 初步版本如下：
# 用法：
from transformers import BertTokenizer, BartForConditionalGeneration, Text2TextGenerationPipeline
tokenizer = BertTokenizer.from_pretrained("fnlp/bart-base-chinese")
model = BartForConditionalGeneration.from_pretrained("fnlp/bart-base-chinese")
text2text_generator = Text2TextGenerationPipeline(model, tokenizer)  
text2text_generator("北京是[MASK]的首都", max_length=50, do_sample=False)

E:\condaenv\py3.10.9\lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

E:\condaenv\py3.10.9\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\86158\.cache\huggingface\hub\models--fnlp--bart-base-chinese. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/259k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.69k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/561M [00:00<?, ?B/s]

[{'generated_text': '北 京 是 中 国 的 首 都'}]

In [8]:
# 加载模型和分词器
model_name = "fnlp/bart-base-chinese"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# 示例：将输入文本进行编码
input_text = "今天的天气非常好，我们去公园散步吧。"
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# 生成文本
output_ids = model.generate(input_ids)
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(output_text)

E:\condaenv\py3.10.9\lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
E:\condaenv\py3.10.9\lib\site-packages\transformers\generation\utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


今 天 的 天 气 非 常 好 ， 我 们 去 公 园 散 步 吧


In [ ]:
# 削减训练数据集，减少运行时间
import json
input_filename="E:\\NLP任务\\生成式任务\\data\\lcsts_tsv\\data1.txt" # 原文件
output_filename="E:\\NLP任务\\生成式任务\\data\\lcsts_tsv\\data1_cutted.txt" # 截取新文件

lines_to_extract=300000

with open(input_filename,'rt',encoding='utf-8') as in_file,open(output_filename,'wt',encoding='utf-8') as out_file:
    line_count=0
    for line in in_file:
        if line_count>=lines_to_extract:
            break
        out_file.write(line)
        line_count+=1
        
print(f'Extracted {line_count} lines and saved to {output_filename}')

In [1]:
# 1.构建数据集：LCSTS，训练集包含200万条数据集，为减少训练时间，将其截取到30万：data1_cutted.txt
# 1.1）加载数据集

import os
import numpy as np
import pandas as pd
import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader

class LCSTS(Dataset):
    def __init__(self,data_file):
        self.data = self.load_data(data_file)
        
    def load_data(self,data_file):
        Data = {}
        with open(data_file,'rt',encoding='utf-8') as f:
            for idx,line in enumerate(f):
                items = line.strip().split('!=!')
                assert len(items) == 2
                Data[idx] = {
                    'title':items[0],
                    'content':items[1],
                }
        return Data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
        return self.data[idx]              

In [2]:
train_data=LCSTS("E:\\NLP任务\\生成式任务\\data\\lcsts_tsv\\data1_cutted.txt")
valid_data=LCSTS("E:\\NLP任务\\生成式任务\\data\\lcsts_tsv\\data2.txt")
test_data=LCSTS("E:\\NLP任务\\生成式任务\\data\\lcsts_tsv\\data3.txt")

In [3]:
# 输出数据集尺寸，并打印一个训练样本

print(f'train set size:{len(train_data)}')
print(f'valid set size:{len(valid_data)}')
print(f'test set size:{len(test_data)}')
print(next(iter(train_data)))

train set size:300000
valid set size:10666
test set size:1106
{'title': '修改后的立法法全文公布', 'content': '新华社受权于18日全文播发修改后的《中华人民共和国立法法》，修改后的立法法分为“总则”“法律”“行政法规”“地方性法规、自治条例和单行条例、规章”“适用与备案审查”“附则”等6章，共计105条。'}


In [4]:
# 1.2)分批，分词，编码
from transformers import BertTokenizer,BartForConditionalGeneration,Text2TextGenerationPipeline
import sentencepiece as spm

model_path = "fnlp/bart-base-chinese"
tokenizer = BertTokenizer.from_pretrained(model_path)# ,local_files_only=True
model = BartForConditionalGeneration.from_pretrained(model_path) # ,local_files_only=True
# text2text_generator = Text2TextGenerationPipeline(model, tokenizer)  
# text2text_generator("北京是[MASK]的首都", max_length=50, do_sample=False)
max_input_length = 512
max_target_length = 64

# 分批函数
def collote_fn(batch_samples):
    batch_inputs,batch_targets = [],[]
    for sample in batch_samples:
        batch_inputs.append(sample['content'])
        batch_targets.append(sample['title'])
    # 源数据
    batch_data = tokenizer(
        batch_inputs,
        padding=True,
        max_length=max_input_length,
        truncation=True,
        return_tensors='pt')
    # 标签数据
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch_targets,
            padding=True,
            max_length=max_target_length,
            truncation=True,
            return_tensors="pt")['input_ids']
        batch_data['decoder_input_ids'] = model.prepare_decoder_input_ids_from_labels(labels)
        end_token_index = torch.where(labels==tokenizer.eos_token_id)[1] # [1]:取列索引
        for idx,end_idx in enumerate(end_token_index):
            labels[idx][end_idx+1:] = -100
        batch_data['labels'] = labels
    return batch_data  

E:\condaenv\py3.10.9\lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
train_dataloader=DataLoader(train_data,batch_size=16,shuffle=True,collate_fn=collote_fn)
valid_dataloader=DataLoader(valid_data,batch_size=16,shuffle=False,collate_fn=collote_fn)
# 注：如果内存不足，batch_size改成16
# 在租卡NVIDIA GeForce RTX 4090 D上出现的错误：
"""
OutOfMemoryError: CUDA out of memory.
Tried to allocate 352.00 MiB (GPU 0; 23.64 GiB total capacity; 22.49 GiB already allocated;
183.75 MiB free; 23.05 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.
See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
"""

'\nOutOfMemoryError: CUDA out of memory.\nTried to allocate 352.00 MiB (GPU 0; 23.64 GiB total capacity; 22.49 GiB already allocated;\n183.75 MiB free; 23.05 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.\nSee documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF\n'

In [11]:
# 打印一个批次数据

batch=next(iter(train_dataloader))
print(batch.keys())
print('batch shape:',{k:v.shape for k,v in batch.items()})
print(batch)

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'decoder_input_ids', 'labels'])
batch shape: {'input_ids': torch.Size([64, 121]), 'token_type_ids': torch.Size([64, 121]), 'attention_mask': torch.Size([64, 121]), 'decoder_input_ids': torch.Size([64, 28]), 'labels': torch.Size([64, 28])}
{'input_ids': tensor([[  101, 10762, 18458,  ...,     0,     0,     0],
        [  101,  5344, 17348,  ...,     0,     0,     0],
        [  101,  8613, 19914,  ...,     0,     0,     0],
        ...,
        [  101,  5344,  7178,  ...,     0,     0,     0],
        [  101, 10317,  5344,  ...,     0,     0,     0],
        [  101,   124, 11225,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
 

E:\condaenv\py3.10.9\lib\site-packages\transformers\tokenization_utils_base.py:3860: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cuda device


In [16]:
# 3.模型训练
# 3.1）训练函数

import torch
from tqdm.auto import tqdm
import os

def train_loop(dataloader,model,optimizer,lr_scheduler,epoch,total_loss):
    model.train()
    model=model.to(device)
    total=0
    
    progress_bar=tqdm(enumerate(dataloader),total=len(dataloader))
    for step,batch_data in progress_bar:
        batch_data=batch_data.to(device)
        # 注意：BertTokenizer不支持token_type_ids，因此这里将具体元素添加进去，而不使用**
        # 而且，有了labels,loss损失才能反向计算
        outputs=model(input_ids=batch_data['input_ids'], labels=batch_data['labels'])
        loss=outputs.loss
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        
        total_loss+=loss.item()
        avg_loss=total_loss/(step+1)
        progress_bar.set_description(f'loss:{avg_loss:>7f}')
    return total_loss

In [14]:
# 3.2) 测试函数：在验证环节中添加评价指标，通常是准确率，召回率，f1,在此任务中使用rouge，包含以上指标
# 注：解码的原因：rouge评价体系所需序列源是文本，而非数字编码
from rouge import Rouge
import random
import numpy as np

rouge=Rouge()
# 0-30：低分
# 30-50：中等
# 50-70：高分
# 70以上：优秀

def test_loop(dataloader,model,mode='Valid'):
    assert mode in ['Valid','Test']
    model=model.to(device)
    model.eval()
    
    preds,labels=[],[]
    for batch_data in dataloader:
        batch_data=batch_data.to(device)
        with torch.no_grad():
            generated_tokens=model.generate( # 1.生成预测
                batch_data['input_ids'],
                attention_mask=batch_data['attention_mask'],
                max_length=max_target_length,
                num_beams=beam_size,# 使用柱搜索
                no_repeat_ngram_size=no_repeat_ngram_size,).cpu().numpy()
        if isinstance(generated_tokens,tuple):
            generated_tokens=generated_tokens[0]
        # 2.对预测解码
        decoded_preds=tokenizer.batch_decode(generated_tokens,skip_special_tokens=True)
        
        label_tokens=batch_data['labels'].cpu().numpy()
        label_tokens=np.where(labels!=-100,label_tokens,tokenizer.pad_token_id)
        decoded_labels=tokenizer.batch_decode(label_tokens,skip_special_tokens=True)
        
        # 用空格连接结果用于匹配rouge格式
        preds+=[' '.join(pred.strip()) for pred in decoded_preds]
        labels+=[' '.join(label.strip()) for label in decoded_labels]      
        
    scores=rouge.get_scores(hyps=preds,refs=labels,avg=True)
    result={key:value['f'] *100 for key,value in scores.items()}
    result['avg']=np.mean(list(result.values()))
    print(f"{mode} Rouge1:{result['rouge-1']:>0.2f} Rouge2:{result['rouge-2']:>0.2f} \
            RougeL:{result['rouge-l']:>0.2f}\n")
    return result

In [17]:
# 3.3) 主循环

from transformers import AutoModelForSeq2SeqLM,AutoTokenizer
from transformers import AdamW,get_scheduler
import random

def seed_everything(seed=1029):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED']=str(seed)
seed_everything(5)

device='cuda' if torch.cuda.is_available() else 'cpu'
print(f'using {device} device')

beam_size=4
no_repeat_ngram_size=2
learning_rate=2e-5
epoch_num=5


optimizer=AdamW(model.parameters(),lr=learning_rate)
lr_scheduler=get_scheduler(
    'linear',
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=epoch_num*len(train_dataloader))

total_loss=0.
best_avg_rouge=0.
for epoch in range(epoch_num):
    print(f'Epoch {epoch+1}/{epoch_num}\n------------------------------------')
    total_loss=train_loop(train_dataloader,model,optimizer,lr_scheduler,epoch+1,total_loss)
    valid_rouge=test_loop(valid_dataloader,model,mode='Valid')
    rouge_avg=valid_rouge['avg']
    if rouge_avg>best_avg_rouge:
        best_avg_rouge=rouge_avg
        print('saving new weights...\n')
        torch.save(model.state_dict(),
                  f'epoch_{epoch+1}_valid_rouge_{rouge_avg:0.4f}_model_weights.bin')
        # 打印验证集评价指标
        print(f'rouge_avg:{rouge_avg}')
        
        # 将验证集指标记录到文件
        with open('rouge_avg.json','a') as f:
            json.dump({'epoch':epoch+1,'rouge':rouge_avg},f)
            f.write('\n')# 确保在文件关闭前执行写入操作

using cuda device
Epoch 1/5
------------------------------------


  0%|          | 0/4688 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 90.00 MiB (GPU 0; 8.00 GiB total capacity; 6.99 GiB already allocated; 0 bytes free; 7.26 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# 4.模型测试
test_data=LCSTS("data/data3.txt")# E:\\NLP任务\\生成式任务\\data\\lcsts_tsv\\data3.txt
test_dataloader=DataLoader(test_data,batch_size=32,shuffle=False,collate_fn=collote_fn)

import json

model.load_state_dict(torch.load('epoch_1_valid_rouge_6.6667_model_weights.bin'))

model.eval()

max_input_length=512
max_target_length=64

with torch.no_grad():
    print('evaluating on test set...')
    sources,preds,labels=[],[],[]
    for batch_data in test_dataloader:
        batch_data=batch_data.to(device)
        generated_tokens=model.generate( # 1.生成预测
            batch_data['input_ids'],
            attention_mak=batch_data['attention_mask'],
            max_length=max_target_length,
            num_beams=4,
            no_repeat_ngram_size=2).cpu().numpy()
        if isinstance(generated_tokens,tuple):
            generated_tokens=generated_tokens[0]
        # 2.对预测解码    
        decoded_preds=tokenizer.batch_decode(generated_tokens,skip_special_tokens=True)
        
        # 转换标签并解码
        label_tokens=batch_data['labels'].cpu().numpy()
        label_tokens=np.where(labels!=-100,label_tokens,tokenizer.pad_token_id)
        decoded_labels=tokenizer.batch_decode(label_tokens,skip_special_tokens=True)
        
        decoded_sources=tokenizer.batch_decode(
            batch_data['input_ids'].cpu().numpy(),
            skip_special_tokens=True,
            use_source_tokenizer=True)
        
        preds+=[' '.join(pred.strip()) for pred in decoded_preds]
        labels+=[' '.join(label.strip()) for label in decoded_labels]
        sources+=[' '.join(source.strip()) for source in decoded_sources]
    scores=rouge.get_scores(
        hyps=preds,refs=labels,avg=True)
    rouges={key:value['f'] *100 for key,value in scores.items()}
    rouges['avg']=np.mean(list(rouges.values()))
    print(f"Test Rouge1: {rouges['rouge-1']:>0.2f} Rouge2: {rouges['rouge-2']:>0.2f} RougeL: {rouges['rouge-l']:>0.2f}\n")
    results=[]
    for source,pred,label in zip(sources,preds,labels):
        results.append({
            'document':source,
            'prediction':pred,
            'summarization':label
        })
    with open('test_data_pred.json','wt',encoding='utf-8') as f:
        for example_result in results:
            f.write(json.dumps(example_result,ensure_ascii=False)+'\n')

In [ ]:
# 在平台上租卡训练时如何指定GPU而非CPU
import torch

# 检查是否有GPU可用
if torch.cuda.is_available():
    device = torch.device("cuda:0") # 如果有多块，可以多指定
    print(f"There are{torch.cuda.device_count()} GPU(s) available.")
    print("Device name:",torch.cuda.get_device_name(0))
    
else:
    device = torch.device("cpu")
    print("No GPU available, using the CPU instead.")
    
# 举例：可以将Tensor或模型移动到指定的设备上  
# 例如，创建一个Tensor并将其移动到GPU上 
tensor = torch.randn(10,10).to(device)


# 或者，在创建模型时直接指定设备  
model = model.to(device) 

In [ ]:
# 显存分配：
# 情况1.如果你是在 Python 脚本或 Jupyter Notebook 中运行的，需要通过 os.environ 来设置环境变量。
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

# 情况2.如果你是在命令行（例如终端或 Anaconda Prompt）中运行 Python 脚本，你可以在运行脚本之前设置这个环境变量：
# bash:
export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128
python your_script.py

# 情况3.如果你是在 Windows 环境下运行，可以使用 set 命令：
set PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128
python your_script.py

In [20]:
# 训练总结：
# 1.训练5轮，每轮训练约25分钟
# 2.租卡参数：NVIDIA GeForce RTX 4090 D
# 3.缓解内存不足：看上面，削减batch_size至16，如果追求大批次，那么16是该资源前提下的下界

In [ ]:
# 模型训练后通常建议手动释放内存，不建议自动释放，手动调节更可控
del model
del optimizer
del dataloader